In [1]:
import os
import sys
module_path = '/home/fatemeh/others_projects/pybam'
if module_path not in sys.path:
    sys.path.append(module_path)

In [10]:
import pysam
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [224]:
print(",".join(["{}U".format(str((2**i)-1)) for i in range(65)]))

0U,1U,3U,7U,15U,31U,63U,127U,255U,511U,1023U,2047U,4095U,8191U,16383U,32767U,65535U,131071U,262143U,524287U,1048575U,2097151U,4194303U,8388607U,16777215U,33554431U,67108863U,134217727U,268435455U,536870911U,1073741823U,2147483647U,4294967295U,8589934591U,17179869183U,34359738367U,68719476735U,137438953471U,274877906943U,549755813887U,1099511627775U,2199023255551U,4398046511103U,8796093022207U,17592186044415U,35184372088831U,70368744177663U,140737488355327U,281474976710655U,562949953421311U,1125899906842623U,2251799813685247U,4503599627370495U,9007199254740991U,18014398509481983U,36028797018963967U,72057594037927935U,144115188075855871U,288230376151711743U,576460752303423487U,1152921504606846975U,2305843009213693951U,4611686018427387903U,9223372036854775807U,18446744073709551615U


### Parse the bam file

In [170]:
def readBam(bamFile):
    vals = []
    st = datetime.now()
    cntr = 1
    infoMap = defaultdict()
    samfile = pysam.AlignmentFile(bamFile, 'rb')
    lscore = 0
    rscore = 0
    lpos = 0
    rpos = 0
#     pysam.AlignedSegment.is_re
    scores = {}
    for read in samfile:        
        rid = read.qname
#         if rid == "SRR2431350.42":
#         print(read)
        if read.is_paired:
            if read.is_unmapped:
                continue 
            aligned_tid = read.reference_name        
            if len(read.get_reference_positions()) == 0:
                print(['broken bam file. no positions set for an aligned read segment: {}, {}' , rid, aligned_tid])
                continue
            if read.is_read1:
                lscore = read.get_tag('AS')
                lpos = read.get_reference_positions()[0]
            if read.is_read2:
                rscore = read.get_tag('AS')
                rpos = read.get_reference_positions()[0]
            next_tid = read.next_reference_name
            next_pos = read.next_reference_start
#             print([rid, aligned_tid, lscore, rscore, lpos, rpos, next_tid, next_pos])
        if read.mate_is_unmapped or (next_tid == aligned_tid and (next_pos == rpos or next_pos == lpos)):
            vals += [[rid, aligned_tid, lscore, rscore, lpos, rpos]]
            cntr+=1
            lscore = 0
            rscore = 0
            lpos = 0
            rpos = 0
            next_pos = -1
        if (cntr % 1000000 == 0):
            print("{} : Read {} lines".format(datetime.now()-st, cntr), end="\r")
    res = pd.DataFrame.from_records(vals)
    print(len(res))
    res.columns = ['readname', 'txp', 'lscore', 'rscore', 'lpos', 'rpos']
    return res

In [177]:
sample="tst"
print('{}/puffalign/{}.bam'.format(root, sample))
tstpf = readBam('{}/puffalign/{}.bam'.format(root, sample))
# tstbt = readBam('{}/bowtie2/{}_dovetail.bam'.format(root, sample))

/mnt/scratch3/fatemeh/pufferfish/outputs/puffalign/tst.bam
1


In [178]:
tstpf

,readname,txp,lscore,rscore,lpos,rpos
0,SRR2431350.913,ENST00000545075,148,148,824,812


In [209]:
# set the sample name and root dir:
root='/mnt/scratch3/fatemeh/pufferfish/outputs'
sample='SRR2431350_1k'

# pufferfish
pf = readBam('{}/puffalign/{}.bam'.format(root, sample))
pf['score'] = pf['lscore']+pf['rscore']
len(pf)

641


641

In [180]:
# bowtie
sample="SRR2431350_1k"
# sample="tst"
bt2 = readBam('{}/bowtie2/{}_dovetail.bam'.format(root, sample))
bt2['score'] = bt2['lscore']+bt2['rscore']
print('\n{}'.format(len(bt2)))
idx = bt2.groupby(['readname'])['score'].transform(max) == bt2['score']
bt = bt2[idx]
print('\n{}'.format(len(bt)))


1129

1129

674


In [201]:
bt2['rscore'].min()

-60

In [198]:
pf[pf['readname'] == 'SRR2431350.247']

,readname,txp,lscore,rscore,lpos,rpos,score
184,SRR2431350.247,ENST00000262193,143,194,31,120,337


In [197]:
bt2[bt2['readname'] == 'SRR2431350.247']

,readname,txp,lscore,rscore,lpos,rpos,score


In [210]:
bt_pf=pd.merge(bt, pf, on = ['readname', 'txp'], how='outer', suffixes=['_bt', '_pf']).fillna(-1)

In [147]:
bt_pf[bt_pf['readname'] == 'SRR2431350.42']

,readname,txp,lscore_bt,rscore_bt,lpos_bt,rpos_bt,score_bt,lscore_pf,rscore_pf,lpos_pf,rpos_pf,score_pf
118,SRR2431350.42,ENST00000369202,-5.0,-6.0,10929.0,11669.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
119,SRR2431350.42,ENST00000369202,-5.0,-6.0,11661.0,12401.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
120,SRR2431350.42,ENST00000369202,-5.0,-6.0,2088.0,2096.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
121,SRR2431350.42,ENST00000369202,-5.0,-6.0,5016.0,5024.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
122,SRR2431350.42,ENST00000369202,-5.0,-6.0,13125.0,13133.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
123,SRR2431350.42,ENST00000369202,-5.0,-6.0,10197.0,10205.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
124,SRR2431350.42,ENST00000369202,-5.0,-6.0,10197.0,10937.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
125,SRR2431350.42,ENST00000369202,-5.0,-6.0,12393.0,12401.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
126,SRR2431350.42,ENST00000369202,-5.0,-6.0,3552.0,3560.0,-11.0,194.0,194.0,13125.0,13133.0,388.0
127,SRR2431350.42,ENST00000369202,-5.0,-6.0,12393.0,13133.0,-11.0,194.0,194.0,13125.0,13133.0,388.0


In [138]:
bt[(bt['readname'] == 'SRR2431350.42') & (bt['txp'] == 'ENST00000432949')]

,readname,txp,lscore,rscore,lpos,rpos,score
16,SRR2431350.42,ENST00000432949,-6,-6,913,921,-12


In [208]:
pd.merge(bt2, bt_pf[bt_pf['lpos_bt'] == -1], on='readname', how='inner')['readname'].unique()

array(['SRR2431350.11', 'SRR2431350.295', 'SRR2431350.189',
       'SRR2431350.260', 'SRR2431350.268', 'SRR2431350.477',
       'SRR2431350.961', 'SRR2431350.913'], dtype=object)

In [202]:
# pufferfish fp
print(len(bt_pf[bt_pf['lpos_bt'] == -1][['readname', 'txp', 'score_pf', 'lpos_pf', 'rpos_pf']]))
bt_pf[bt_pf['lpos_bt'] == -1][['readname', 'txp', 'lscore_pf', 'rscore_pf','score_pf', 'lpos_pf', 'rpos_pf']]

37


,readname,txp,lscore_pf,rscore_pf,score_pf,lpos_pf,rpos_pf
674,SRR2431350.11,ENST00000264033,132.0,146.0,278.0,8116.0,8071.0
675,SRR2431350.66,ENST00000600213,146.0,131.0,277.0,526.0,304.0
676,SRR2431350.180,ENST00000366956,134.0,200.0,334.0,5352.0,5408.0
677,SRR2431350.189,ENST00000600213,153.0,141.0,294.0,232.0,879.0
678,SRR2431350.216,ENST00000375627,139.0,155.0,294.0,552.0,412.0
679,SRR2431350.226,ENST00000400202,200.0,135.0,335.0,7514.0,7489.0
680,SRR2431350.226,ENST00000400199,200.0,135.0,335.0,7405.0,7380.0
681,SRR2431350.247,ENST00000262193,143.0,194.0,337.0,31.0,120.0
682,SRR2431350.260,ENST00000379254,200.0,0.0,200.0,12.0,0.0
683,SRR2431350.260,ENST00000379251,200.0,162.0,362.0,12.0,0.0


In [113]:
# pufferfish fn
bt_pf[bt_pf['lpos_pf'] == -1][['readname', 'txp', 'score_bt', 'lpos_bt', 'rpos_bt']]

,readname,txp,score_bt,lpos_bt,rpos_bt
4,SRR2431350.5,ENST00000535534,-101.0,3134.0,3151.0
5,SRR2431350.5,ENST00000355610,-101.0,3395.0,3412.0
8,SRR2431350.11,ENST00000296292,-84.0,4613.0,4568.0
28,SRR2431350.161,ENST00000600213,-84.0,457.0,389.0
29,SRR2431350.162,ENST00000262942,-55.0,1294.0,1384.0
44,SRR2431350.116,ENST00000544932,-76.0,88.0,361.0
82,SRR2431350.36,ENST00000600213,-71.0,601.0,773.0
105,SRR2431350.43,ENST00000392994,0.0,38.0,0.0
121,SRR2431350.324,ENST00000222553,-115.0,1118.0,1108.0
152,SRR2431350.295,ENST00000543500,-80.0,1076.0,1075.0


In [211]:
# pufferfish fp
print(len(bt_pf[bt_pf['lpos_bt'] == -1][['readname', 'txp', 'score_pf', 'lpos_pf', 'rpos_pf']]))
bt_pf[bt_pf['lpos_bt'] == -1][['readname', 'txp', 'lscore_pf', 'rscore_pf','score_pf', 'lpos_pf', 'rpos_pf']]

19


,readname,txp,lscore_pf,rscore_pf,score_pf,lpos_pf,rpos_pf
674,SRR2431350.11,ENST00000264033,132.0,146.0,278.0,8116.0,8071.0
675,SRR2431350.189,ENST00000600213,146.0,134.0,280.0,228.0,879.0
676,SRR2431350.247,ENST00000262193,131.0,194.0,325.0,23.0,120.0
677,SRR2431350.260,ENST00000379254,200.0,0.0,200.0,12.0,0.0
678,SRR2431350.260,ENST00000379251,200.0,140.0,340.0,12.0,0.0
679,SRR2431350.268,ENST00000429578,162.0,149.0,311.0,1457.0,1555.0
680,SRR2431350.295,ENST00000604952,152.0,152.0,304.0,1260.0,1259.0
681,SRR2431350.307,ENST00000522835,200.0,136.0,336.0,601.0,492.0
682,SRR2431350.339,ENST00000433673,134.0,140.0,274.0,897.0,933.0
683,SRR2431350.339,ENST00000445217,134.0,140.0,274.0,1706.0,1742.0


In [212]:
# pufferfish fn
bt_pf[bt_pf['lpos_pf'] == -1][['readname', 'txp', 'score_bt', 'lpos_bt', 'rpos_bt']]

,readname,txp,score_bt,lpos_bt,rpos_bt
4,SRR2431350.5,ENST00000535534,-108.0,3147.0,3151.0
5,SRR2431350.5,ENST00000355610,-108.0,3408.0,3412.0
8,SRR2431350.11,ENST00000296292,-102.0,4613.0,4568.0
72,SRR2431350.161,ENST00000600213,-96.0,457.0,389.0
73,SRR2431350.162,ENST00000262942,-57.0,1294.0,1384.0
81,SRR2431350.36,ENST00000600213,-83.0,609.0,773.0
105,SRR2431350.43,ENST00000392994,0.0,38.0,0.0
162,SRR2431350.295,ENST00000543500,-90.0,1076.0,1075.0
182,SRR2431350.227,ENST00000600213,-72.0,348.0,441.0
229,SRR2431350.189,ENST00000536684,-83.0,299.0,946.0
